In [136]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from cv2 import imread, imwrite
from ipywidgets import interact, widgets
from pathlib import Path
from os import listdir
from os.path import isfile, join

In [137]:
p = Path.home()
p = p/'OneDrive'/'Documentos'/'GitHub'/'jupyter'/'jupiter-notebooks'/'src'/'riscos'/'teste'

In [138]:
filenames = [f for f in listdir(p) if isfile(join(p, f))]
imgs = dict()

for filename in filenames:
    filepath = p/filename
    try:
        imgs[filepath.name] = imread(str(filepath))
    except FileNotFoundError:
        pass

In [139]:
dfs = dict()
left_edge_px = dict()
right_edge_px = dict()
high = dict()
low = dict()
width = dict()

for key, img in imgs.items():
    dfs[key] = pd.DataFrame({'Blue': imgs[key][:,:,2].sum(axis = 0),
                       'Red': imgs[key][:,:,0].sum(axis = 0),
                       'Green': imgs[key][:,:,1].sum(axis = 0)})
    high[key] = max(dfs[key]['Red'])
    low[key] = min(dfs[key]['Red'])
    
for key in list(imgs.keys()):
    df = dfs[key].reset_index()
    mask = df['index']<1500
    left_edge_px[key] = df.loc[mask & (df['Red']>=0.9*np.mean((high[key],low[key])))].index.min()
    right_edge_px[key] = df.loc[mask & (df['Red']>=0.9*np.mean((high[key],low[key])))].index.max()
    width[key] = 1.262626*(right_edge_px[key]-left_edge_px[key])

In [140]:
def plot_colors(key):
    fig = px.line(dfs[key])
    fig.show()

In [141]:
1.262626*(right_edge_px[key]-left_edge_px[key])

496.212018

In [142]:
def plot_overlay(key):
    path = p/key
    # Create figure
    fig = go.Figure()

    # Constants
    img_width = 2048
    img_height = 1532
    scale_factor = 0.3

    # Add invisible scatter trace.
    # This trace is added to help the autoresize logic work.
    fig.add_trace(
        go.Scatter(
            x=[0, img_width * scale_factor],
            y=[0, img_height * scale_factor],
            mode="markers",
            marker_opacity=0
        )
    )

    # Configure axes
    fig.update_xaxes(
        visible=False,
        range=[0, img_width * scale_factor]
    )

    fig.update_yaxes(
        visible=False,
        range=[0, img_height * scale_factor],
        # the scaleanchor attribute ensures that the aspect ratio stays constant
        scaleanchor="x"
    )

    # Add image
    fig.add_layout_image(
        dict(
            x=0,
            sizex=img_width * scale_factor,
            y=img_height * scale_factor,
            sizey=img_height * scale_factor,
            xref="x",
            yref="y",
            opacity=1.0,
            layer="below",
            sizing="stretch",
            source='src/riscos/teste/{}'.format(key)[:-4]+'.jpg')
    )
    fig.add_vline(x = 0.3*left_edge_px[key], line_color = 'red')
    fig.add_vline(x = 0.3*right_edge_px[key], line_color = 'red')

    # Configure other layout
    fig.update_layout(
        width=img_width * scale_factor,
        height=img_height * scale_factor,
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
    )

    # Disable the autosize on double click because it adds unwanted margins around the image
    # More detail: https://plotly.com/python/configuration-options/
    fig.show(config={'doubleClick': 'reset'})

In [143]:
interact(plot_colors, key = list(dfs.keys()))

interactive(children=(Dropdown(description='key', options=('PU+1GE-11-a.jpg', 'PU+1GE-11-b.jpg', 'PU+1GE-11-c.…

<function __main__.plot_colors(key)>

In [144]:
interact(plot_overlay, key = list(imgs.keys()))

interactive(children=(Dropdown(description='key', options=('PU+1GE-11-a.jpg', 'PU+1GE-11-b.jpg', 'PU+1GE-11-c.…

<function __main__.plot_overlay(key)>

In [145]:
width

{'PU+1GE-11-a.jpg': 280.302972,
 'PU+1GE-11-b.jpg': 253.787826,
 'PU+1GE-11-c.jpg': 252.5252,
 'PU+1GE-12-a.jpg': 421.717084,
 'PU+1GE-12-b.jpg': 405.302946,
 'PU+1GE-12-c.jpg': 450.757482,
 'PU+1GE-6-b.jpg': 439.393848,
 'PU+1GE-6-c.jpg': 452.020108,
 'PU+1GE-7-a.jpg': 441.9191,
 'PU+1GE-7-b.jpg': 421.717084,
 'PU+1GE-7-c.jpg': 440.656474,
 'PU+1GE-8-a.jpg': 338.38376800000003,
 'PU+1GE-8-b.jpg': 345.959524,
 'PU+1GE-8-c.jpg': 362.373662,
 'PU+2GE-10-a.jpg': 282.828224,
 'PU+2GE-10-b.jpg': 279.040346,
 'PU+2GE-10-c.jpg': 284.09085,
 'PU+2GE-11-a.jpg': 378.7878,
 'PU+2GE-11-b.jpg': 378.7878,
 'PU+2GE-11-c.jpg': 366.16154,
 'PU+2GE-12-a.jpg': 474.74737600000003,
 'PU+2GE-7-a.jpg': 357.32315800000003,
 'PU+2GE-7-b.jpg': 359.84841,
 'PU+2GE-7-c.jpg': 357.32315800000003,
 'PU+2GE-8-a.jpg': 446.969604,
 'PU+2GE-8-b.jpg': 436.868596,
 'PU+2GE-8-c.jpg': 449.494856,
 'PU+2GE-9-a.jpg': 311.868622,
 'PU+2GE-9-b.jpg': 303.03024,
 'PU+2GE-9-c.jpg': 300.504988,
 'PU+4EG_50x_13_a.jpg': 276.515094000

In [146]:
width_measured = pd.read_excel(r'C:\Users\Caio\OneDrive\Documentos\Python\Results\Excel\Resultados Largura de Desgaste.xlsx')

In [147]:
width_measured['width'] = width_measured[['Largura 1','Largura 2','Largura 3']].fillna(0).apply(np.mean, axis = 'columns')

In [148]:
width_measured

,Foto,PU,EG,Posição,Ensaio,Largura 1,Largura 2,Largura 3,Força Normal,Frequência,width
0,PU+1GE-1-a,99,1,Meio,1,397.46,394.95,396.21,8.00,4,396.206667
1,PU+1GE-1-b,99,1,Ponta,1,408.87,439.23,NaN,8.00,4,282.700000
2,PU+1GE-1-c,99,1,Ponta,1,426.56,406.31,NaN,8.00,4,277.623333
3,PU+1GE-2-a,99,1,Ponta,2,338.00,327.84,312.68,5.00,2,326.173333
4,PU+1GE-2-b,99,1,Meio,2,302.53,315.27,315.34,5.00,2,311.046667
...,...,...,...,...,...,...,...,...,...,...,...
145,PU+C_2GE-3-b,98,2,Meio,15,367.08,362.02,367.08,5.17,3,365.393333
146,PU+C_2GE-3-c,98,2,Ponta,15,362.01,362.02,369.60,5.17,3,364.543333
147,PU+C_2GE-4-a,98,2,Ponta,16,353.15,351.89,355.68,5.17,3,353.573333
148,PU+C_2GE-4-b,98,2,Meio,16,346.82,354.42,349.36,5.17,3,350.200000


In [149]:
std_error = 0
n = 0
error = dict()
for key, value in width.items():
    mask = width_measured['Foto']==key[:-4]
    error[key] = value - float(width_measured.loc[mask, 'width'])
    std_error += error[key]**2
    n += 1
std_dev = np.sqrt(std_error/(n-1))

In [150]:
error

{'PU+1GE-11-a.jpg': -7.6136946666666745,
 'PU+1GE-11-b.jpg': -15.045507333333376,
 'PU+1GE-11-c.jpg': -15.471466666666657,
 'PU+1GE-12-a.jpg': -11.362915999999984,
 'PU+1GE-12-b.jpg': -13.400387333333356,
 'PU+1GE-12-c.jpg': 33.334148666666636,
 'PU+1GE-6-b.jpg': -5.8761519999999905,
 'PU+1GE-6-c.jpg': -7.049892,
 'PU+1GE-7-a.jpg': -3.4142333333333,
 'PU+1GE-7-b.jpg': -7.076249333333351,
 'PU+1GE-7-c.jpg': -6.8368593333333365,
 'PU+1GE-8-a.jpg': -6.899565333333271,
 'PU+1GE-8-b.jpg': 4.532857333333311,
 'PU+1GE-8-c.jpg': 3.980328666666651,
 'PU+2GE-10-a.jpg': 3.7815573333332964,
 'PU+2GE-10-b.jpg': 4.183679333333373,
 'PU+2GE-10-c.jpg': -9.715816666666683,
 'PU+2GE-11-a.jpg': 1.4211333333333869,
 'PU+2GE-11-b.jpg': 8.431133333333378,
 'PU+2GE-11-c.jpg': -10.90179333333333,
 'PU+2GE-12-a.jpg': 4.267376000000013,
 'PU+2GE-7-a.jpg': -15.560175333333348,
 'PU+2GE-7-b.jpg': -1.2349233333333132,
 'PU+2GE-7-c.jpg': -8.816842000000008,
 'PU+2GE-8-a.jpg': -8.187062666666577,
 'PU+2GE-8-b.jpg': 

In [151]:
std_dev

12.986321500326737